# Project Three Notebook Example

For your final notebook, feel free to duplicate this notebook and edit as needed. 


## Load Some Stuff

This is where we load libraires and the like so we can do what we need. If you get an error saying a module is not loaded, open a new terminal/cmd line and try running: `pip install [module name]`. 

In [2]:
import os
try:
    inputFunc = raw_input
except NameError:
    inputFunc = input

import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import numpy as np
 
import seaborn as sns
from statsmodels.formula.api import ols

from sklearn import linear_model
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from patsy import dmatrices

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

import random



# Custom functions

def evaluate(pred, labels_test):
    acc = accuracy_score(pred, labels_test)
    print ("Accuracey: %s"%acc)
    tn, fp, fn, tp = confusion_matrix(labels_test, pred).ravel()

    recall = tp / (tp + fp)
    percision = tp / (tp + fn)
    f1 = (2 / ((1/recall)+(1/percision)))

    print ("")
    print ("True Negatives: %s"%tn)
    print ("False Positives: %s"%fp)
    print ("False Negatives: %s"%fn)
    print ("True Positives: %s"%tp)
    print ("Recall: %s"%recall)
    print ("Precision: %s"%percision)
    print ("F1 Score: %s"%f1)

def plot_bound(Z_val,data,col1,col2,binary):
    # Z-val equals "Yes" value. E.g., "Y" or "1". 
    # data equals df
    # col1 and col2 defines which colums to use from data
    # Plot binary decision boundary. 
    # For this, we will assign a color to each
    # point in the mesh [x_min, m_max]x[y_min, y_max].
    
    x_min = float(data.iloc[:,[col1]].min())-float(data.iloc[:,[col1]].min())*0.10 
    x_max = float(data.iloc[:,[col1]].max()+float(data.iloc[:,[col1]].min())*0.10)
    y_min = 0.0; 
    y_max = float(training.iloc[:,[col2]].max())+float(training.iloc[:,[col2]].max())*0.10
    h_x = (x_max-x_min)/100  # step size in the mesh
    h_y = (y_max-y_min)/100  # step size in the mesh
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h_x), np.arange(y_min, y_max, h_y))
    if binary == 1:
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])   
        Z = np.where(Z=="Y",1,0)
    else:
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.pcolormesh(xx, yy, Z)
    plt.show()

## Data Cleaning

Here we load the data we collected and get it all ready to feed to our statistical model(s). That is, we are trying to make a table with one **target** column and one or more **features**. Here I'm loading happiness.csv from: https://data.somervillema.gov/Happiness/Somerville-Happiness-Survey-responses-2011-2013-20/w898-3dfm Note: you can find information on the data elements at this link. 


In [3]:
# Load and peek at your data. Change the file name as needed. 
raw_data_df = pd.read_csv('911.csv', parse_dates=[0]) 
raw_data_df.head()

,Year,Month,Total,BPD,BFD,EMS
0,2010-01-01,11,2603,1969,259,375
1,2010-01-01,11,2414,1852,206,356
2,2010-01-01,11,2487,1872,235,380
3,2010-01-01,11,2328,1695,241,392
4,2010-01-01,11,2478,1915,225,338


In [4]:
# You can explore unique entires by stating the column and using .unique() like this:
print(raw_data_df["Total"].unique())
print(raw_data_df["Month"].unique())

[2603 2414 2487 2328 2478 2297 2113 2542 2523 2496 2108 2665 2443 2136 2367
 2500 2463 2533 2631 2324 2020 2431 2489 2217 1413 1935 2080 1862 2318 2400
 2497 2483 2577 2176 1985 2198 2306 2394 2447 2248 1910 2311 2289 2334 2232
 2504 2267 1959 2646 2355 2456 2302 1787 1086 1533 2228 2215 2237 2291 2517
 1796 2258 2348 2195 2449 1878 1779 2229 2410 2975 2383 1895 1992 2526 2653
 2614 2200 2308 2213 2561 2684 2432 2530 2787 2492 2226 2616 2507 2675 2833
 2506 2230 2750 2593 2582 2584 2642 2359 2196 2574 2571 2832 2404 2011 2079
 2336 2397 2335 2345 1886 2409 2625 2454 2471 2553 2520 2175 2479 2403 2617
 2030 2442 2354 2635 2836 2325 2339 2575 2285 1993 2480 2588 2485 2509 2392
 2234 2448 2619 2583 2590 2615 2559 2199 2649 2462 2294 2589 2651 2194 2387
 2301 2424 1978 2634 2726 2989 2562 2630 2606 2691 2946 2560 2641 2624 3256
 2993 2499 2116 2426 2436 2643 2799 2925 2511 2550 2850 2965 2908 3034 2441
 2287 2764 2820 2740 2868 2676 2365 2704 2849 2738 2190 2690 2748 2707 2922
 2259 2752 2

In [5]:
# You can explore rows with a specific value like so
raw_data_df[raw_data_df["Total"]>3200].head() # remove .head() to see all entires

,Year,Month,Total,BPD,BFD,EMS
192,2011-01-01,5,3256,2672,218,366
430,2012-01-01,1,3253,2646,241,366
526,2012-01-01,4,3209,2565,239,405
532,2012-01-01,4,3347,2356,316,675
576,2012-01-01,5,3210,2622,209,379


In [6]:
# You can count the number of rows like so
print("Size of entire table: %s "%len(raw_data_df))
print("Size of entires matching filter: %s "%len(raw_data_df[raw_data_df["Total"]>3200]))

Size of entire table: 1268 
Size of entires matching filter: 56 


In [7]:
# You can invert a match by using the 'not equal' evaluation.
print("Size of entires matching filter: %s "%len(raw_data_df[raw_data_df["Total"]<3200]))
raw_data_df[raw_data_df["Total"]<3200].head()

Size of entires matching filter: 1212 


,Year,Month,Total,BPD,BFD,EMS
0,2010-01-01,11,2603,1969,259,375
1,2010-01-01,11,2414,1852,206,356
2,2010-01-01,11,2487,1872,235,380
3,2010-01-01,11,2328,1695,241,392
4,2010-01-01,11,2478,1915,225,338


In [8]:
# You can make a new table from your filtered rows like so
processed_data_df = raw_data_df[raw_data_df["Total"]>3200]
processed_data_df = processed_data_df[processed_data_df["BPD"]>2000]
# Note how I filtered first on raw_data_df and then on processed_data_df

# So how many entires are there?
print("Size of entire table: %s "%len(processed_data_df))
# Let's peak at the table.
processed_data_df.head()

Size of entire table: 56 


,Year,Month,Total,BPD,BFD,EMS
192,2011-01-01,5,3256,2672,218,366
430,2012-01-01,1,3253,2646,241,366
526,2012-01-01,4,3209,2565,239,405
532,2012-01-01,4,3347,2356,316,675
576,2012-01-01,5,3210,2622,209,379


In [ ]:
# You can replace values in a column based on logic like so
# Note: I used the unique values found above to inform my logic.
# That is, I took the unique text lables and translated them into numbers.
# It's clear that different surveys had different buckets. So I probably 
# sould limit myself to years using the same metrics, but for our purposes
# I'm just going to run with a quick and dirty translation. 

processed_data_df.loc[processed_data_df['income'] == 'Less than $10,000', 'income'] = 5000 
processed_data_df.loc[processed_data_df['income'] == '10,000 - $19,999', 'income'] = 15000
processed_data_df.loc[processed_data_df['income'] == '$10,000 to $24,999', 'income'] = 17500
processed_data_df.loc[processed_data_df['income'] == '20,000 - $29,999', 'income'] = 25000
processed_data_df.loc[processed_data_df['income'] == '$25,000 to $49,999', 'income'] = 37500
processed_data_df.loc[processed_data_df['income'] == '30,000 - $39,999', 'income'] = 35000
processed_data_df.loc[processed_data_df['income'] == '40,000 - $49,999', 'income'] = 45000
processed_data_df.loc[processed_data_df['income'] == '50,000 - $59,999', 'income'] = 55000
processed_data_df.loc[processed_data_df['income'] == '$50,000 to $74,999', 'income'] = 62500
processed_data_df.loc[processed_data_df['income'] == '60,000 - $69,999', 'income'] = 65000
processed_data_df.loc[processed_data_df['income'] == '70,000 - $79,999', 'income'] = 75000
processed_data_df.loc[processed_data_df['income'] == '$75,000 to $99,999', 'income'] = 87500
processed_data_df.loc[processed_data_df['income'] == '80,000 - $89,999', 'income'] = 85000
processed_data_df.loc[processed_data_df['income'] == '90,000 - $99,999', 'income'] = 95000
processed_data_df.loc[processed_data_df['income'] == '100,000 and up', 'income'] = 100000
processed_data_df.loc[processed_data_df['income'] == '$100,000 to $149,999', 'income'] = 125000 
processed_data_df.loc[processed_data_df['income'] == '$150,000 or more', 'income'] = 150000

processed_data_df.loc[processed_data_df['age'] == '18-21', 'age'] = 24
processed_data_df.loc[processed_data_df['age'] == '18-24', 'age'] = 21
processed_data_df.loc[processed_data_df['age'] == '22-25', 'age'] = 23.5
processed_data_df.loc[processed_data_df['age'] == '25-34', 'age'] = 29.5
processed_data_df.loc[processed_data_df['age'] == '26-30', 'age'] = 28
processed_data_df.loc[processed_data_df['age'] == '31-40', 'age'] = 35.5
processed_data_df.loc[processed_data_df['age'] == '35-44', 'age'] = 39.5
processed_data_df.loc[processed_data_df['age'] == '41-50', 'age'] = 46.5
processed_data_df.loc[processed_data_df['age'] == '45-54', 'age'] = 48
processed_data_df.loc[processed_data_df['age'] == '51-60', 'age'] = 55.5
processed_data_df.loc[processed_data_df['age'] == '55-64', 'age'] = 58
processed_data_df.loc[processed_data_df['age'] == '61+', 'age'] = 61
processed_data_df.loc[processed_data_df['age'] == '65-74', 'age'] = 69.5
processed_data_df.loc[processed_data_df['age'] == '75 or older', 'age'] = 75
processed_data_df.head()


In [ ]:
# To make sure all of your columns are stored as numbers, use the pd.to_numeric method like so.
processed_data_df = processed_data_df.apply(pd.to_numeric, errors='coerce')
# errors='coerce' will set things that can't be converted to numbers to NaN
# so you'll want to drop these like so.
processed_data_df = processed_data_df.dropna()
processed_data_df.head()

In [ ]:
# You can add a columns like so.
processed_data_df["happy_Y_N"] = "N"
processed_data_df["satisfied_Y_N"] = "N"
processed_data_df.head()

In [ ]:
# And then you can customize these new coulmns using the same method as above. 
processed_data_df.loc[processed_data_df['happy'] >= 5, 'happy_Y_N'] = "Y"
processed_data_df.loc[processed_data_df['satisfied'] >= 5, 'satisfied_Y_N'] = "Y"
processed_data_df.head()

In [ ]:
# I'm now going to make a set of tables to be used in training some models
# The first set will be for linear regressions where the traget is numeric.
# Happiness
happy_lin_df = processed_data_df[[
                               'happy', 
                               'age', 
                               'income'
                               ]].copy()
happy_lin_df.head()

In [ ]:
# Satisfaction
sat_lin_df = processed_data_df[[
                               'satisfied', 
                               'age', 
                               'income'
                               ]].copy()
sat_lin_df.head()

In [ ]:
# The second set will be for classifiers where the target is a class.
# Happiness
happy_class_df = processed_data_df[[
                               'happy_Y_N', 
                               'age', 
                               'income'
                               ]].copy()
happy_class_df.head()

In [ ]:
# Satisfaction
sat_class_df = processed_data_df[[
                               'satisfied_Y_N', 
                               'age', 
                               'income'
                               ]].copy()
sat_class_df.head()

## Taining and Validation

Above I created four datasets worth exploring: 
- **`happy_lin_df`**. The data needed to access *happiness* along a continuous variable.
- **`sat_lin_df`**. The data needed to access *satisfaction* along a continuous variable.
- **`happy_class_df`**. The data needed to access *happiness* as a categorical variable.
- **`sat_class_df`**. The data needed to access *satisfaction* as a categorical variable.

Let's take them each in turn. 

## happy_lin_df



In [ ]:
data = happy_lin_df

data = data[data["happy"]<=10]

holdout = data.sample(frac=0.05)
training = data.loc[~data.index.isin(holdout.index)]

In [ ]:
sns.lmplot(x="age", y="happy", data=training, x_estimator=np.mean, order=1)

In [ ]:
sns.lmplot(x="income", y="happy", data=training, x_estimator=np.mean, order=1)

In [ ]:
model = ols("happy ~ age + income", training).fit()
#model = ols("happy ~ age + income + np.power(age, 2) + np.power(income, 2)", training).fit()
model.summary()

In [ ]:
# Rerun with SciKitLearn because it's easy to check accuracy
features_train = training.drop("happy", axis=1).as_matrix(columns=None)
labels_train = training["happy"].as_matrix(columns=None)

features_test = holdout.drop("happy", axis=1).as_matrix(columns=None)
labels_test = holdout["happy"].as_matrix(columns=None)

lm = linear_model.LinearRegression()
clf = lm.fit(features_train, labels_train)
pred = clf.predict(features_test)
accuracy = metrics.r2_score(labels_test, pred)
print("R squared:",lm.score(features_train,labels_train))
print("Accuracy:",accuracy)

## sat_lin_df

In [ ]:
data = sat_lin_df

data = data[data["satisfied"]<=10]

holdout = data.sample(frac=0.05)
training = data.loc[~data.index.isin(holdout.index)]

In [ ]:
sns.lmplot(x="age", y="satisfied", data=training, x_estimator=np.mean, order=1)

In [ ]:
sns.lmplot(x="income", y="satisfied", data=training, x_estimator=np.mean, order=1)

In [ ]:
model = ols("satisfied ~ age + income", training).fit()
model.summary()

In [ ]:
# Rerun with SciKitLearn because it's easy to check accuracy

features_train = training.drop("satisfied", axis=1).as_matrix(columns=None)
labels_train = training["satisfied"].as_matrix(columns=None)

features_test = holdout.drop("satisfied", axis=1).as_matrix(columns=None)
labels_test = holdout["satisfied"].as_matrix(columns=None)

lm = linear_model.LinearRegression()
clf = lm.fit(features_train, labels_train)
pred = clf.predict(features_test)
accuracy = metrics.r2_score(labels_test, pred)
print("R squared:",lm.score(features_train,labels_train))
print("Accuracy:",accuracy)

## happy_class_df

In [ ]:
data = happy_class_df
holdout = data.sample(frac=0.05)
training = data.loc[~data.index.isin(holdout.index)]

# Define the target (y) and feature(s) (X)
features_train = training.drop("happy_Y_N", axis=1).as_matrix(columns=None)
labels_train = training["happy_Y_N"].as_matrix(columns=None)

features_test = holdout.drop("happy_Y_N", axis=1).as_matrix(columns=None)
labels_test = holdout["happy_Y_N"].as_matrix(columns=None)

# What percentage of the time is target Y?
print("Percentage of Ys: %s\n"%(len(data[data["happy_Y_N"]=="Y"])/len(data)))

#### initial visualization
feature_1_no = [features_test[ii][0] for ii in range(0, len(features_test)) if labels_test[ii]=="N"]
feature_2_no = [features_test[ii][1] for ii in range(0, len(features_test)) if labels_test[ii]=="N"]
feature_1_yes = [features_test[ii][0] for ii in range(0, len(features_test)) if labels_test[ii]=="Y"]
feature_2_yes = [features_test[ii][1] for ii in range(0, len(features_test)) if labels_test[ii]=="Y"]
plt.scatter(feature_1_yes, feature_2_yes, color = "g", label="Happy")
plt.scatter(feature_1_no, feature_2_no, color = "r", label="Unhappy")
plt.legend()
plt.xlabel("age")
plt.ylabel("income")
plt.show()


In [ ]:
# Logistic Regression
model = LogisticRegression(fit_intercept = False, C = 1e9)
clf = model.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("Logistic Regression")
evaluate(pred, labels_test)  
plot_bound("Y",holdout,1,2,0)


# Test some spot
x_test = 70
y_test = 160000
print("")
print(clf.predict([[x_test,y_test]])[0])
print(clf.predict_proba([[x_test,y_test]])[0][1])
print("")

from sklearn import tree
clf = tree.DecisionTreeClassifier(min_samples_split=40)
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("\nDecision Tree")
evaluate(pred, labels_test)
plot_bound("Y",holdout,1,2,0)



from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("Random Forest")
evaluate(pred, labels_test)  
plot_bound("Y",holdout,1,2,0)


from sklearn.svm import SVC
clf = SVC(kernel="rbf",probability=True)
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("SVM")
evaluate(pred, labels_test)  
#plot_bound("Y",holdout,1,2,0) # plot doesn't work with SVM

## sat_class_df

In [ ]:
data = sat_class_df
holdout = data.sample(frac=0.05)
training = data.loc[~data.index.isin(holdout.index)]

# Define the target (y) and feature(s) (X)
features_train = training.drop("satisfied_Y_N", axis=1).as_matrix(columns=None)
labels_train = training["satisfied_Y_N"].as_matrix(columns=None)

features_test = holdout.drop("satisfied_Y_N", axis=1).as_matrix(columns=None)
labels_test = holdout["satisfied_Y_N"].as_matrix(columns=None)

# What percentage of the time is target Y?
print("Percentage of Ys: %s\n"%(len(data[data["satisfied_Y_N"]=="Y"])/len(data)))

#### initial visualization
feature_1_no = [features_test[ii][0] for ii in range(0, len(features_test)) if labels_test[ii]=="N"]
feature_2_no = [features_test[ii][1] for ii in range(0, len(features_test)) if labels_test[ii]=="N"]
feature_1_yes = [features_test[ii][0] for ii in range(0, len(features_test)) if labels_test[ii]=="Y"]
feature_2_yes = [features_test[ii][1] for ii in range(0, len(features_test)) if labels_test[ii]=="Y"]
plt.scatter(feature_1_yes, feature_2_yes, color = "g", label="Happy")
plt.scatter(feature_1_no, feature_2_no, color = "r", label="Unhappy")
plt.legend()
plt.xlabel("age")
plt.ylabel("income")
plt.show()

In [ ]:
# Logistic Regression
model = LogisticRegression(fit_intercept = False, C = 1e9)
clf = model.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("Logistic Regression")
evaluate(pred, labels_test)  
plot_bound("Y",holdout,1,2,0)


# Test some spot
x_test = 70
y_test = 160000
print("")
print(clf.predict([[x_test,y_test]])[0])
print(clf.predict_proba([[x_test,y_test]])[0][1])
print("")

from sklearn import tree
clf = tree.DecisionTreeClassifier(min_samples_split=40)
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("\nDecision Tree")
evaluate(pred, labels_test)
plot_bound("Y",holdout,1,2,0)



from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("Random Forest")
evaluate(pred, labels_test)  
plot_bound("Y",holdout,1,2,0)


from sklearn.svm import SVC
clf = SVC(kernel="rbf",probability=True)
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("SVM")
evaluate(pred, labels_test)  
#plot_bound("Y",holdout,1,2,0) # plot doesn't work with SVM